# XGBoost - Predicción usando dataset_final
Este notebook carga `data/dataset_final.csv`, realiza el split (70/15/15) sin barajar, entrena XGBoost con los hiperparámetros usados en `models.ipynb`, evalúa en validación y test, y muestra una predicción de ejemplo.

In [1]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


# Ruta del dataset procesado
data_dir = Path.cwd().parent.parent / "data"
df = pd.read_csv(data_dir / "dataset_final.csv")

# Definición de features y target
TARGET = "BTC-Close_next_day"
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Alinear por seguridad (si el último 'next_day' no existe o por consistencia)
X = X.iloc[:-1]
y = y.iloc[:-1]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (2043, 286)
y shape: (2043,)


In [2]:
from xgboost import XGBRegressor
# Guardamos una fila de ejemplo para "predecir el siguiente día" (mismo patrón que RF)
x_last_row = X.head(1).copy()
y_last_row = y.head(1).copy()
print("x_last_row shape:", x_last_row.shape)
print("y_last_row:", y_last_row.values)

# Split 70/15/15 sin shuffle para respetar temporalidad
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=False
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

x_last_row shape: (1, 286)
y_last_row: [122391.]
Train: (1430, 286) Val: (306, 286) Test: (307, 286)


In [3]:
# Modelo XGBoost con los hiperparámetros de models.ipynb
xgb = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.5,
    reg_alpha=0.5,
    min_child_weight=3,
    random_state=42,
    eval_metric="mae",
    early_stopping_rounds=50
)

# Entrenamiento con conjunto de validación para early stopping
xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# Validación
y_val_pred = xgb.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)
print("1- XGBOOST VALIDATION RESULTS")
print(f"MAE: {mae_val:.4f}")
print(f"R2: {r2_val:.4f}")

# Test
y_test_pred = xgb.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
print("2- XGBOOST TEST RESULTS")
print(f"MAE: {mae_test:.4f}")
print(f"R2: {r2_test:.4f}")

1- XGBOOST VALIDATION RESULTS
MAE: 1472.8417
R2: 0.4031
2- XGBOOST TEST RESULTS
MAE: 1087.8868
R2: 0.7693


In [4]:
# Predicción de ejemplo con la primera fila como proxy del 'siguiente día' (mismo patrón que RF)
y_predicted_next = xgb.predict(x_last_row)
print(f" Valor real anterior: {y_last_row.values[0]:,.2f} USD")
print(f" Predicción del precio de BTC para el siguiente día: {y_predicted_next[0]:,.2f} USD")
print(f" Diferencia: {abs(y_predicted_next[0] - y_last_row.values[0]):,.2f} USD")

 Valor real anterior: 122,391.00 USD
 Predicción del precio de BTC para el siguiente día: 122,372.95 USD
 Diferencia: 18.05 USD
